In [1]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Makes using jupyter notebook on laptops much easier
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
spotify_df = pd.read_csv('dataset.csv')

In [4]:
spotify_df.head()

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4


In [5]:
spotify_df.dtypes

Unnamed: 0            int64
artist_name          object
track_name           object
track_id             object
popularity            int64
year                  int64
genre                object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms           int64
time_signature        int64
dtype: object

In [6]:
spotify_df['genre'].values[325991]

'drum-and-bass'

Confirm the output is a string

In [7]:
type(spotify_df['artist_name'].values[93587])

str

In [8]:
spotify_df[spotify_df['track_name']=='Adore You']

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
93587,93587,Miley Cyrus,Adore You,5AnCLGg35ziFOloEnXK4uu,69,2013,pop,0.583,0.6550,0,-5.407,1,0.0315,0.1110,0.000004,0.1130,0.201,119.759,278747,4
406373,406373,Harry Styles,Adore You,3jjujdWJ72nww5eGnfs2E7,84,2019,pop,0.676,0.7710,8,-3.675,1,0.0483,0.0237,0.000007,0.1020,0.569,99.048,207133,4
461551,461551,Ellery Hughes,Adore You,4lREOFz1zdTy3SoEpQPdTm,36,2020,piano,0.447,0.0775,0,-20.554,0,0.0552,0.9880,0.920000,0.1060,0.071,99.335,204625,4
472900,472900,Jessie Ware,Adore You,1TZpVXWfeFsnoTz6idYnkW,42,2020,soul,0.669,0.6680,10,-11.039,0,0.0391,0.2990,0.199000,0.1050,0.201,106.989,226373,4
771068,882587,Anointed,Adore You,5azYoFDdV2YaedTPO3a8p8,6,2003,gospel,0.380,0.8200,6,-8.039,0,0.0492,0.0819,0.000092,0.2070,0.614,183.559,239067,4
1110262,1395772,Calvin Richardson,Adore You,4ddYCgyLgsP98RdCs97mpA,15,2010,soul,0.660,0.5060,2,-7.326,1,0.0950,0.2980,0.000002,0.0886,0.534,76.917,205107,4


In [9]:
spotify_df['artist_plus_song'] = spotify_df.apply(lambda row: str(row['artist_name'])+str(row['track_name']),axis = 1)

In [10]:
spotify_df[spotify_df['track_name']=='Adore You']

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_plus_song
93587,93587,Miley Cyrus,Adore You,5AnCLGg35ziFOloEnXK4uu,69,2013,pop,0.583,0.6550,0,...,1,0.0315,0.1110,0.000004,0.1130,0.201,119.759,278747,4,Miley CyrusAdore You
406373,406373,Harry Styles,Adore You,3jjujdWJ72nww5eGnfs2E7,84,2019,pop,0.676,0.7710,8,...,1,0.0483,0.0237,0.000007,0.1020,0.569,99.048,207133,4,Harry StylesAdore You
461551,461551,Ellery Hughes,Adore You,4lREOFz1zdTy3SoEpQPdTm,36,2020,piano,0.447,0.0775,0,...,0,0.0552,0.9880,0.920000,0.1060,0.071,99.335,204625,4,Ellery HughesAdore You
472900,472900,Jessie Ware,Adore You,1TZpVXWfeFsnoTz6idYnkW,42,2020,soul,0.669,0.6680,10,...,0,0.0391,0.2990,0.199000,0.1050,0.201,106.989,226373,4,Jessie WareAdore You
771068,882587,Anointed,Adore You,5azYoFDdV2YaedTPO3a8p8,6,2003,gospel,0.380,0.8200,6,...,0,0.0492,0.0819,0.000092,0.2070,0.614,183.559,239067,4,AnointedAdore You
1110262,1395772,Calvin Richardson,Adore You,4ddYCgyLgsP98RdCs97mpA,15,2010,soul,0.660,0.5060,2,...,1,0.0950,0.2980,0.000002,0.0886,0.534,76.917,205107,4,Calvin RichardsonAdore You


In [11]:
artists_exploded = spotify_df[['artist_name','track_id']].explode('artist_name')

In [12]:
artists_exploded

,artist_name,track_id
0,Jason Mraz,53QF56cjZA9RTuuMZDrSA6
1,Jason Mraz,1s8tP3jP4GZcyHDsjvw218
2,Joshua Hyslop,7BRCa8MPiyuvr2VU3O9W0F
3,Boyce Avenue,63wsZUhUZLlh1OsyrZq7sz
4,Andrew Belle,6nXIYClvJAfi6ujLiKqEq8
...,...,...
1159759,Nicola Conte,0m27F0IGHLGAWhqd6ccYst
1159760,Nicola Conte,6er9p611eHEcUCU50j7D57
1159761,Amon Tobin,7jsMMqxy1tt0rH5FzYcZTQ
1159762,Peace Orchestra,77lA1InUaXztuRk2vOzD1S


In [13]:
artist_genres = spotify_df.groupby('track_id')['genre'].apply(list).reset_index()

In [14]:
artist_genres

,track_id,genre
0,0001piYJu94Ec4hJFytG5G,[goth]
1,0005VnpISGYLSGrXg9TEJS,[heavy-metal]
2,0006PpnEOPCCxToaXEEFnQ,[rock-n-roll]
3,00075E86IyrXpuZcdibi60,[gospel]
4,0009mEWM7HILVo4VZYtqwc,[minimal-techno]
...,...,...
1159759,7zzqlH9rTiZgUxYiObgmdX,[disco]
1159760,7zzrXzx0A7C7RriCE8tVIe,[dub]
1159761,7zzxVXohk4jjxt3ur6mrjI,[black-metal]
1159762,7zzyGbrAbA425AYf4SDGC8,[power-pop]


Normalize float variables

In [15]:
float_cols = spotify_df.dtypes[spotify_df.dtypes == 'float64'].index.values

In [16]:
ohe_cols = 'popularity'

In [17]:
spotify_df['popularity'].describe()

count    1.159764e+06
mean     1.838312e+01
std      1.588554e+01
min      0.000000e+00
25%      5.000000e+00
50%      1.500000e+01
75%      2.900000e+01
max      1.000000e+02
Name: popularity, dtype: float64

In [18]:
spotify_df['popularity_red'] = spotify_df['popularity'].apply(lambda x: int(x/5))

In [19]:
spotify_df.head()

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_plus_song,popularity_red
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,...,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3,Jason MrazI Won't Give Up,13
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,...,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4,Jason Mraz93 Million Miles,10
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,...,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4,Joshua HyslopDo Not Let Me Go,11
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,...,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4,Boyce AvenueFast Car,11
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,...,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4,Andrew BelleSky's Still Blue,10


In [20]:
def ohe_prep(df, column, new_name):
    '''
    create OHE features of a specific column
    '''
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)
    return tf_df

In [21]:
def create_feature_set(df, float_cols):
    '''
    Process spotify df to create a final set of features that will be used to generate recommendations
    '''
    
    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genre'])
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.reset_index(drop = True, inplace=True)

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    year_ohe = ohe_prep(df, 'year','year') * 0.5
    popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)
     
    #add song id
    final['track_id']=df['track_id'].values
    
    return final

In [22]:
complete_feature_set = create_feature_set(spotify_df, float_cols=float_cols)

In [23]:
complete_feature_set.head()

,genre|acoustic,genre|afrobeat,genre|age,genre|alt,genre|ambient,genre|and,genre|bass,genre|black,genre|blues,genre|breakbeat,...,year|2015,year|2016,year|2017,year|2018,year|2019,year|2020,year|2021,year|2022,year|2023,track_id
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53QF56cjZA9RTuuMZDrSA6
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1s8tP3jP4GZcyHDsjvw218
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7BRCa8MPiyuvr2VU3O9W0F
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63wsZUhUZLlh1OsyrZq7sz
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6nXIYClvJAfi6ujLiKqEq8


Connect Spotify API

In [24]:
client_id = '172e70c6e61a400b94acd7f2b5c2fe23'
client_secret = '6296812cf4694a988e1c46e0ffc83ec8'

In [25]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [26]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager = auth_manager)

In [27]:
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret= client_secret, redirect_uri='http://localhost:4111/')

In [28]:
sp = spotipy.Spotify(auth=token)

In [29]:
id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    # list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [30]:
id_name

{'Discover Weekly': '37i9dQZEVXcTbiOEPHPCG3',
 'ANJOONA': '5Sy3itcBtCt57EGordXBC1',
 'pepitas': '0InMNqpDsVdE2TNESYYrPt',
 "songs that don't age": '4wTk2KYIb9koQNE64oMUVP',
 'hard seltzer': '5CaNrGhM4waIbFrKfPQi0n',
 'subWOOFer': '3I4zKf8J0PAeqP3HBuZGtm',
 'madhappy': '3RWynTIvC2YIGYrgz69iJZ',
 'melodub': '58YQZF6KAMr8U5Er5JpFMa',
 'background music': '515jU4YyqKtv1PBF27npxv',
 'hause': '4RhKpGrdGpA4RsAC7yDs06',
 'we outside': '2ZJjCxOGhYZz2sFte83NZL',
 'classiq': '1BmF14OviS7DovOhVyV4zX',
 'prog tronce': '0HPPdNCTgpycMigQ3xd7aL',
 'psytrance': '0nAKvrCAEUHPhw55flvb4F',
 'anjunastuff': '4rKfcXUgHPaqIFeWOufHuS',
 'feelin güd': '5EF6c1CEMn8St34jJlPvyq',
 'funky beats yo': '1cGYqyJ2dZZfVCnDYDvOqy',
 'Video Music': '5sl2GhUQ52QNKRb8KmY8j1',
 'fc.': '3O3My5afwhtCakWPEVp7W8',
 'edm': '2VVMGybKW4H8HBz9areWWs',
 'teen angst': '0CbcgKIEoSxX8Wr8b1Bwrl',
 'its a trap': '3WElV8pELWy2LdO77amWZ9',
 'Songs that excite the white folk': '1YOgHsPyptYJOe8p63Kyhe',
 'DBT House Warming': '4TR9QiLNOK86FQouY

In [44]:
def create_necessary_outputs(playlist_name,id_dic, df):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify datafram
        
    Returns: 
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE KAGGLE DATASET
    """
    
    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['track_id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [45]:
playlist_funky = create_necessary_outputs('funky beats yo', id_name, spotify_df)

In [46]:
playlist_funky

,artist,name,id,url,date_added
34,bbno$,edamame - Diplo Remix,1fvEC5ksBfgjGWVNP9eJ2t,https://i.scdn.co/image/ab67616d00001e02a11f2b...,2021-12-17 22:51:48+00:00
33,Alison Wonderland,Anything - Body Ocean Remix,4piGD6lbuj57rxausi0brN,https://i.scdn.co/image/ab67616d00001e02544262...,2021-06-18 20:30:44+00:00
30,Bob Moses,Listen To Me - Cassian Remix,7BOQEVXrxhHUpR0RJBm4t1,https://i.scdn.co/image/ab67616d00001e02801f6d...,2021-06-18 20:13:14+00:00
27,Kaskade,Fakin It (feat. Ofelia K),02r2uE8nEHpJAvFFttQxqh,https://i.scdn.co/image/ab67616d00001e02bf1831...,2020-08-25 23:35:06+00:00
25,Kaskade,On Your Mind,5IpIl6c14WCJhjlIl6poVD,https://i.scdn.co/image/ab67616d00001e0287a438...,2020-08-25 23:33:22+00:00
24,ZHU,Stay Closer,55uvoAKSRcZZmoKU853yd1,https://i.scdn.co/image/ab67616d00001e02a5aa7b...,2020-08-25 23:32:04+00:00
23,ZHU,Paradise Awaits,7w0RvC8LO5xpj3klbbS9rG,https://i.scdn.co/image/ab67616d00001e02a5aa7b...,2020-08-25 23:31:59+00:00
22,ZHU,Faded,2GQEM9JuHu30sGFvRYeCxz,https://i.scdn.co/image/ab67616d00001e02a5aa7b...,2020-08-25 23:31:41+00:00
21,ZHU,Automatic,1NB0VPwAw6Rx8b9qvDKB5M,https://i.scdn.co/image/ab67616d00001e0264efcb...,2020-08-25 23:31:29+00:00
20,CamelPhat,Cola - ZHU Remix,0TjqjAJpzv3EHrsEelLQGZ,https://i.scdn.co/image/ab67616d00001e02cb597a...,2020-08-25 23:31:16+00:00


In [47]:
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    """ 
    Summarize a user's playlist into a single vector

    Parameters: 
        complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['track_id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added']], left_on = 'track_id', right_on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['track_id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)
    
    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
        
    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    #print(playlist_feature_set_weighted.iloc[:,:-4].columns)
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,1))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    #playlist_feature_set_weighted_final['id'] = playlist_feature_set['id']
    
    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [48]:
complete_feature_set_playlist_vector_funky, complete_feature_set_nonplaylist_funky = generate_playlist_feature(complete_feature_set, playlist_funky, 1.09)

In [49]:
complete_feature_set_playlist_vector_funky.shape

(140,)

In [50]:
def generate_playlist_recos(df, features, nonplaylist_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_20: Top 20 recommendations for that playlist
    """
    
    non_playlist_df = df[df['track_id'].isin(nonplaylist_features['track_id'].values)]
    # print(features.iloc[:-1].values)
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('track_id', axis = 1).values, features.iloc[:-1].values.reshape(1, -1))[:,0]
    non_playlist_df_top_20 = non_playlist_df.sort_values('sim',ascending = False).head(20)
    non_playlist_df_top_20['url'] = non_playlist_df_top_20['track_id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_20

In [51]:
funky_top20 = generate_playlist_recos(spotify_df, complete_feature_set_playlist_vector_funky, complete_feature_set_nonplaylist_funky)

In [52]:
funky_top20

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_plus_song,popularity_red,sim,url
231040,231040,DENNIS,Automaticamente,6AaoEX192rJ6o3UFwG43sV,48,2016,edm,0.918,0.907,0,...,0.000003,0.0637,0.839,132.076,187839,4,DENNISAutomaticamente,9,0.841245,https://i.scdn.co/image/ab67616d00001e027c03cf...
384158,384158,Landis,Hit The Flow,7r2VuLH3NqOu0bXF976eFY,47,2019,edm,0.817,0.987,6,...,0.043200,0.3300,0.643,128.002,151875,4,LandisHit The Flow,9,0.840256,https://i.scdn.co/image/ab67616d00001e025e490e...
230944,230944,Garmiani,Voodoo (feat. Walshy Fire),4I6Oju4HroXhn54k8Kkxh8,45,2016,edm,0.855,0.994,5,...,0.000010,0.6350,0.858,128.003,144609,4,GarmianiVoodoo (feat. Walshy Fire),9,0.839897,https://i.scdn.co/image/ab67616d00001e02b2c942...
384305,384305,Jax Jones,House Work (feat. Mike Dunn & MNEK),4N1NX1NM6hOQDx11jJC8cn,45,2019,edm,0.765,0.946,0,...,0.319000,0.0413,0.777,123.976,157373,4,Jax JonesHouse Work (feat. Mike Dunn & MNEK),9,0.839491,https://i.scdn.co/image/ab67616d00001e02bb3c16...
384146,384146,Anti Up,Concentrate,5oXkfuwL7HieiHzvmWsaGo,38,2019,edm,0.851,0.866,6,...,0.415000,0.1110,0.609,125.994,188089,4,Anti UpConcentrate,7,0.839249,https://i.scdn.co/image/ab67616d00001e024779ac...
230733,230733,RetroVision,Puzzle,6AP4OWS9khErpiQpkPdQx6,48,2016,edm,0.777,0.850,0,...,0.559000,0.3440,0.692,127.992,166008,4,RetroVisionPuzzle,9,0.838969,https://i.scdn.co/image/ab67616d00001e029badbe...
384250,384250,SIDEPIECE,Wanna See You,69EEyee0qJltlXswEaOaNS,36,2019,edm,0.891,0.937,9,...,0.042500,0.0604,0.685,124.974,249934,4,SIDEPIECEWanna See You,7,0.838872,https://i.scdn.co/image/ab67616d00001e021caaa3...
384228,384228,Anti Up,Right Now,1N6WOpIIxFoMW1xCRruxPY,36,2019,edm,0.810,0.885,1,...,0.360000,0.0404,0.634,125.998,215238,4,Anti UpRight Now,7,0.838558,https://i.scdn.co/image/ab67616d00001e02be3f15...
384145,384145,Boston Bun,Don't Wanna Dance,6b56B8Ew3d0WDnd4Vq01Gh,45,2019,edm,0.797,0.827,5,...,0.101000,0.2880,0.680,121.979,169402,4,Boston BunDon't Wanna Dance,9,0.838402,https://i.scdn.co/image/ab67616d00001e023de81c...
230997,230997,Valentino Khan,Slam Dunk (feat. Kstylis),2kSijCecfo9ywjCVpaFhcr,36,2016,edm,0.717,0.979,9,...,0.000156,0.0984,0.515,149.997,217600,4,Valentino KhanSlam Dunk (feat. Kstylis),7,0.838331,https://i.scdn.co/image/ab67616d00001e02e07393...


test2:

In [54]:
playlist_we_outside = create_necessary_outputs('we outside', id_name, spotify_df)

In [55]:
complete_feature_set_playlist_vector_we_outside, complete_feature_set_nonplaylist_we_outside = generate_playlist_feature(complete_feature_set, playlist_we_outside, 1.09)

In [56]:
outside_top20 = generate_playlist_recos(spotify_df, complete_feature_set_playlist_vector_we_outside, complete_feature_set_nonplaylist_we_outside)

In [57]:
outside_top20

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_plus_song,popularity_red,sim,url
284958,284958,$uicideboy$,Obey the Walrus,6nQM3VyF8fQCRiIiZsvnMp,60,2017,hip-hop,0.851,0.807,5,...,0.000000,0.1180,0.394,149.204,141819,4,$uicideboy$Obey the Walrus,12,0.926193,https://i.scdn.co/image/ab67616d00001e02e37269...
284928,284928,Logic,Everybody,7cGFbx7MP0H23iHZTZpqMM,62,2017,hip-hop,0.885,0.940,1,...,0.000000,0.0675,0.773,110.005,162347,4,LogicEverybody,12,0.926098,https://i.scdn.co/image/ab67616d00001e02cfdf40...
285339,285339,Yandel,Como Antes (feat. Wisin),5YzA563GXTuwQaRq24z1k5,60,2017,hip-hop,0.813,0.876,1,...,0.000000,0.3940,0.746,94.987,210773,4,YandelComo Antes (feat. Wisin),12,0.925962,https://i.scdn.co/image/ab67616d00001e02b54d1c...
284945,284945,Playboi Carti,dothatshit!,1KzNsOkpQthVwpCJrADJEQ,61,2017,hip-hop,0.746,0.744,9,...,0.000000,0.2790,0.452,162.964,184964,4,Playboi Cartidothatshit!,12,0.925933,https://i.scdn.co/image/ab67616d00001e02e31a27...
285278,285278,Wisin,Vacaciones,1rXojdsUqqxGj2WCmJGWHP,62,2017,hip-hop,0.777,0.911,6,...,0.000000,0.2570,0.704,96.017,238813,4,WisinVacaciones,12,0.925921,https://i.scdn.co/image/ab67616d00001e02c7524a...
284949,284949,$uicideboy$,Slip on a Banana Clip,0XwE208D5TzDLgCJUaYCFv,60,2017,hip-hop,0.757,0.794,0,...,0.000000,0.3150,0.448,140.027,137195,4,$uicideboy$Slip on a Banana Clip,12,0.925867,https://i.scdn.co/image/ab67616d00001e02e37269...
284962,284962,JAY-Z,The Story of O.J.,6JpN5w95em8SODPiM7W2PH,62,2017,hip-hop,0.741,0.718,7,...,0.000002,0.2300,0.576,165.848,231760,4,JAY-ZThe Story of O.J.,12,0.925853,https://i.scdn.co/image/ab67616d00001e027513c4...
285024,285024,Huncho Jack,Motorcycle Patches,7g7raxdQpiLZT7aOlib4S1,60,2017,hip-hop,0.868,0.657,8,...,0.000000,0.0995,0.436,162.867,191410,4,Huncho JackMotorcycle Patches,12,0.925645,https://i.scdn.co/image/ab67616d00001e02631973...
284869,284869,Migos,Slippery (feat. Gucci Mane),0TBgMKD86t1VP3IzhxCoUl,68,2017,hip-hop,0.920,0.675,1,...,0.000000,0.1040,0.749,141.967,304041,4,MigosSlippery (feat. Gucci Mane),13,0.925633,https://i.scdn.co/image/ab67616d00001e026275ae...
285645,285645,Guru Randhawa,Suit Suit,2mwBSO58y92EYmEFpDfmS7,62,2017,hip-hop,0.767,0.869,2,...,0.000000,0.2210,0.751,98.208,190132,4,Guru RandhawaSuit Suit,12,0.925574,https://i.scdn.co/image/ab67616d00001e02b092db...
